In [1]:
import urllib3
import json
import traceback

webhook_url = 'https://hooks.slack.com/services/SECRET_TOKEN'

# Send Slack notification based on the given message
def slack_notification(message):
    try:
        slack_message = {'text': message}

        http = urllib3.PoolManager()
        response = http.request('POST',
                                webhook_url,
                                body = json.dumps(slack_message),
                                headers = {'Content-Type': 'application/json'},
                                retries = False)
    except:
        traceback.print_exc()

    return True

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
data = pd.read_csv("covid-data.csv")

In [4]:
data.head()

,Unnamed: 0,date,state,fips,cases,deaths
0,0,1/21/2020,Washington,53,1,0
1,1,1/22/2020,Washington,53,1,0
2,2,1/23/2020,Washington,53,1,0
3,3,1/24/2020,Illinois,17,1,0
4,4,1/24/2020,Washington,53,1,0


In [5]:
data.shape

(8154, 6)

In [6]:
data = data.drop(columns=['Unnamed: 0'])

In [7]:
data.head()

,date,state,fips,cases,deaths
0,1/21/2020,Washington,53,1,0
1,1/22/2020,Washington,53,1,0
2,1/23/2020,Washington,53,1,0
3,1/24/2020,Illinois,17,1,0
4,1/24/2020,Washington,53,1,0


In [8]:
data['state'].unique()

array(['Washington', 'Illinois', 'California', 'Arizona', 'Massachusetts',
       'Wisconsin', 'Texas', 'Nebraska', 'Utah', 'Oregon', 'Florida',
       'New York', 'Rhode Island', 'Georgia', 'New Hampshire',
       'North Carolina', 'New Jersey', 'Colorado', 'Maryland', 'Nevada',
       'Tennessee', 'Hawaii', 'Indiana', 'Kentucky', 'Minnesota',
       'Oklahoma', 'Pennsylvania', 'South Carolina',
       'District of Columbia', 'Kansas', 'Missouri', 'Vermont',
       'Virginia', 'Connecticut', 'Iowa', 'Louisiana', 'Ohio', 'Michigan',
       'South Dakota', 'Arkansas', 'Delaware', 'Mississippi',
       'New Mexico', 'North Dakota', 'Wyoming', 'Alaska', 'Maine',
       'Alabama', 'Idaho', 'Montana', 'Puerto Rico', 'Virgin Islands',
       'Guam', 'West Virginia', 'Northern Mariana Islands'], dtype=object)

In [9]:
import datetime
data = data.assign(date = pd.to_datetime(data.date, format='%m/%d/%Y'))

In [10]:
death_toll_statewise = data.groupby([data.date.dt.month,'state'])['deaths'].sum().reset_index()

In [11]:
death_toll_statewise.rename(columns={'date': 'month'}, inplace=True)

In [30]:
death_toll_statewise.head()

,month,state,deaths
0,1,Arizona,0
1,1,California,0
2,1,Illinois,0
3,1,Washington,0
4,2,Arizona,0


In [38]:
data_with_month_name = death_toll_statewise.assign(
    month_name = lambda dataframe: dataframe['month'].map(
        lambda month: 'January' if month == 1 
        else 'February'  if month == 2 
        else 'March'  if month == 3
        else 'April'  if month == 4 
        else 'May'  if month == 5  
        else 'June'  if month == 6 
        else 'July'  if month == 7  
        else 'August'  if month == 8 
        else 'September'  if month == 9  
        else 'October'  if month == 10 
        else 'November'  if month == 11  
        else 'December'  if month == 12 
        else 'Invalid Month') 
)

In [40]:
data_with_month_name = data_with_month_name.drop(columns=['month'])

In [55]:
unique_months = data_with_month_name['month_name'].unique()
for i in range(3,7):
    req_data = data_with_month_name[data_with_month_name['month_name'] == unique_months[i]].nlargest(3,'deaths')
    json_data = req_data[['month_name','state','deaths']].to_json(orient='records')[1:-1].replace('},{', '} {')
    slack_notification(json_data)